<a href="https://colab.research.google.com/github/arafMustavi/Chatbot/blob/master/ChatBot_Using_Pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Setup of Environment

In [1]:
import nltk
nltk.download('punkt')
from nltk.stem.porter import PorterStemmer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Tokenizer and Stemmer

In [2]:
def tokenize(sentence):
  return nltk.word_tokenize(sentence)

stemmer = PorterStemmer()

def stem_word(word):
  word = word.lower()
  return stemmer.stem(word)
  

Load Data and Train

In [3]:
import json

with open('/content/sample_data/intents.json','r') as f:
  intents = json.load(f)

In [4]:
all_words = []
tags = []
xy = []

for intent in intents['intents']:
  tag = intent['tag']
  tags.append(tag)
  for pattern in intent['patterns']:
    tokens = tokenize(pattern)
    all_words.extend(tokens)
    xy.append((tokens,tag))

In [5]:
ignore_words = ["?" , "," , "!" , "." , "(" , ")" ]

In [6]:
all_words = [stem_word(word) for word in all_words if word not in ignore_words ]

all_words = sorted(set(all_words))
tags = sorted(set(tags))

Train Data

In [7]:
import numpy as np
X_train = []
Y_train = []
X_train = np.array(X_train)
Y_train = np.array(Y_train)


In [8]:
def bag_of_words(tokenized_sentence,all_words):
  tokenized_sentence = [stem_word(word) for word in tokenized_sentence]
  bag = np.zeros(len(all_words) , dtype = np.float32)

  for (idx,w) in enumerate(all_words):
    if w in tokenized_sentence:
      bag[idx] = 1.0
  
  return bag  

In [9]:
for (pattern_sentence,tag) in xy:
  bag = bag_of_words(pattern_sentence,all_words)
  X_train=np.append(X_train,bag)

  label = tags.index(tag)
  Y_train=np.append(Y_train,label)

PyTorch Begins

In [10]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset,DataLoader

In [11]:
class chatDataset(Dataset):
  def __init__(self):
    self.n_samples = X_train.size
    self.X_data = X_train
    self.Y_data = Y_train

    def __getitem__(self,idx):
      return self.X_data[idx],self.Y_data[idx]
    
    def __len__(self):
      return self.n_samples

In [12]:
#Hyperparameters
batch_size = 8
hidden_size = 8
output_size = len(tags)
input_size = X_train.size
learning_rate = 0.001
num_epochs = 1000

dataset = chatDataset()
train_loader = DataLoader(dataset = dataset, batch_size = batch_size, shuffle = True , num_workers = 2)

TypeError: ignored

# Model
A feed forward Neural Network with two hidden layer


In [ ]:
class NeuralNet(nn.Module):
  def __init__(self , input_size , hidden_size , num_classes):
      super(NeuralNet,self).__init__()
      self.l1=nn.Linear(input_size,hidden_size)
      self.l2=nn.Linear(hidden_size,hidden_size)
      self.l3=nn.Linear(hidden_size,num_classes)

      self.relu = nn.ReLU()

  def forward(self,x):
    out = self.l1(x)
    out = self.relu(out)
    
    out = self.l2(out)
    out = self.relu(out)
    
    out = self.l3(out)
    
    return out
    


Train 

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model =  NeuralNet(input_size, hidden_size , output_size).to(device)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters() , lr = learning_rate)

for epoch in range(num_epochs):
  for (words , labels) in train_loader:
    words = words.to(device)
    labels = labels.to(device)

    #Forward
    outputs = model(words)
    loss = criterion(outputs, labels)

    #Backward and Optimizer Step
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
  if (epoch+1)%100 == 0 :
    print(f'epoch {epoch+1}/{num_epochs}, loss = {loss.item():.4f}')
print(f'final loss , loss = {loss.item():.4f}')